In [10]:
using Graphs
using Plots
import Statistics: mean
include("../mpdbp.jl")
include("../exact/montecarlo.jl")
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [11]:
q = q_glauber
T = 10
N = 16
k = 3
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
h = zeros(N)
J = ones(ne(g))
β = 1.0
ising = Ising(g, J, h, β)
p⁰ = map(1:N) do i
    r = 0.75
    [r, 1-r]
end
ϕ = [[ones(2) for t in 1:T] for i in 1:N]

gl = Glauber(ising, p⁰, ϕ);

Run BP

In [12]:
ε = 1e-2
bp = mpdbp(gl, d=1)
cb = CB_BP(bp)
iterate!(bp, maxiter=50; ε, cb, tol=1e-3)

Running MPdBP: iter 9 	 Time: 1:24:19
  Δ:  0.0903854786793079

LoadError: InterruptException:

In [13]:
m_bp = magnetizations(bp; ε)
avg_mag = mean(m_bp)
pl = plot(0:T, avg_mag, m=:o, xlabel="time", ylabel="average mag", label="BP")

LoadError: InterruptException:

Run Monte Carlo

In [14]:
gs = sample(gl, nsamples=10^4, showprogress=true);

LoadError: InterruptException:

In [15]:
m = magnetizations(gs)
mm = mean(m, dims=2) |> vec
plot!(pl, 0:T, mm, label="Monte Carlo", m=:square)

LoadError: UndefVarError: pl not defined